### We will classify the text into 7 discreet categories (Sentiment Analysis and TEXT_CLASSIFICATION) using a fine-tuned LLM (DistillROBERTA) from HF
-  Categories:
    - `anger` `disgust` `fear` `joy` `sadness` `surprise` `neutral`

In [1]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')

In [3]:
from transformers import pipeline

classifier = pipeline("text-classification",
                      model = "j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "cuda")

classifier("I love this!")

Device set to use cuda


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.0057645998895168304},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119939973577857},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


- We have a choice get the sentiment of a whole description or break it down into sentences and get sentiment of each sentence.

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

- We can see from the ist example running sentiment analysis on a single sentence we may lose information.

In [5]:
# Prediction on the whole description
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548401117324829},
  {'label': 'neutral', 'score': 0.16985271871089935},
  {'label': 'sadness', 'score': 0.11640920490026474},
  {'label': 'surprise', 'score': 0.02070067822933197},
  {'label': 'disgust', 'score': 0.019100727513432503},
  {'label': 'joy', 'score': 0.015161407180130482},
  {'label': 'anger', 'score': 0.003935148939490318}]]

- The prediction comes out to be `fearfull` but looking at the description above eg in line *Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details*, it doesn't sound `fearful` but `joyous`.
- The correct way seems to be to break down the description into individual sentences and classfiy on that basis.

In [6]:
classifier(books["description"][0].split("."))  # Infer each sentence

[[{'label': 'surprise', 'score': 0.7296029329299927},
  {'label': 'neutral', 'score': 0.14038559794425964},
  {'label': 'fear', 'score': 0.06816216558218002},
  {'label': 'joy', 'score': 0.04794234409928322},
  {'label': 'anger', 'score': 0.009156346321105957},
  {'label': 'disgust', 'score': 0.0026284747291356325},
  {'label': 'sadness', 'score': 0.0021221598144620657}],
 [{'label': 'neutral', 'score': 0.44937023520469666},
  {'label': 'disgust', 'score': 0.27359238266944885},
  {'label': 'joy', 'score': 0.10908287763595581},
  {'label': 'sadness', 'score': 0.09362705796957016},
  {'label': 'anger', 'score': 0.040478311479091644},
  {'label': 'surprise', 'score': 0.026970146223902702},
  {'label': 'fear', 'score': 0.006879040505737066}],
 [{'label': 'neutral', 'score': 0.6462163925170898},
  {'label': 'sadness', 'score': 0.24273303151130676},
  {'label': 'disgust', 'score': 0.04342260584235191},
  {'label': 'surprise', 'score': 0.02830052748322487},
  {'label': 'joy', 'score': 0.01421

- We can see a lot of varieties of emotions per sentence therefore a true representation of the whole description.

In [7]:
# Let's see for an example sentence which class has higher probs, to have a better view
sentences = books["description"][0].split(".")
predictions = classifier(sentences)


In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296029329299927},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'fear', 'score': 0.06816216558218002},
 {'label': 'joy', 'score': 0.04794234409928322},
 {'label': 'anger', 'score': 0.009156346321105957},
 {'label': 'disgust', 'score': 0.0026284747291356325},
 {'label': 'sadness', 'score': 0.0021221598144620657}]

In [10]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [11]:
lhpredictions[3]

[{'label': 'fear', 'score': 0.928168535232544},
 {'label': 'anger', 'score': 0.03219063952565193},
 {'label': 'neutral', 'score': 0.012808633036911488},
 {'label': 'sadness', 'score': 0.008756852708756924},
 {'label': 'surprise', 'score': 0.008597886189818382},
 {'label': 'disgust', 'score': 0.008431801572442055},
 {'label': 'joy', 'score': 0.001045580138452351}]

- We see now that for each sentence the classifier is giving expected labels based on the tone.
- So this also complicates our task, since we have multiple emotions for single books.
- To solve this we will add 7 cols to our dataset and for each book take the emotion with the highest probability.

In [12]:
predictions

[[{'label': 'surprise', 'score': 0.7296029329299927},
  {'label': 'neutral', 'score': 0.14038559794425964},
  {'label': 'fear', 'score': 0.06816216558218002},
  {'label': 'joy', 'score': 0.04794234409928322},
  {'label': 'anger', 'score': 0.009156346321105957},
  {'label': 'disgust', 'score': 0.0026284747291356325},
  {'label': 'sadness', 'score': 0.0021221598144620657}],
 [{'label': 'neutral', 'score': 0.44937023520469666},
  {'label': 'disgust', 'score': 0.27359238266944885},
  {'label': 'joy', 'score': 0.10908287763595581},
  {'label': 'sadness', 'score': 0.09362705796957016},
  {'label': 'anger', 'score': 0.040478311479091644},
  {'label': 'surprise', 'score': 0.026970146223902702},
  {'label': 'fear', 'score': 0.006879040505737066}],
 [{'label': 'neutral', 'score': 0.6462163925170898},
  {'label': 'sadness', 'score': 0.24273303151130676},
  {'label': 'disgust', 'score': 0.04342260584235191},
  {'label': 'surprise', 'score': 0.02830052748322487},
  {'label': 'joy', 'score': 0.01421

In [14]:
# Sort labels in decreasing order by emotion prob scores
sorted(predictions[0], key = lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156346321105957},
 {'label': 'disgust', 'score': 0.0026284747291356325},
 {'label': 'fear', 'score': 0.06816216558218002},
 {'label': 'joy', 'score': 0.04794234409928322},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'sadness', 'score': 0.0021221598144620657},
 {'label': 'surprise', 'score': 0.7296029329299927}]

In [21]:
import  numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"] # cols of new df
isbn = [] # key for merging to df
emotions_scores = {label: [] for label in emotion_labels}    # Extract all scores for every desc. for each label

def calculate_max_emotion_score(predictions):
    """ Extract maximum emotion probability: for each emotion, for each description

    :param predictions:
    :return: dict: maximum emotion probability
    """

    # Extract all predictions for a single description
    per_emotion_scores = {label: [] for label in emotion_labels}

    for prediction in predictions:
        sorted_predictions = sorted(prediction, key = lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return  {label: np.max(scores) for label, scores in per_emotion_scores.items()}

- Now for each desc. we have a dictionary containing the max prob. for each of the diff. emotion labels

In [22]:
for i in range(10):
    isbn.append(books["isbn13"][i])  # add individual isbns of each book to the list
    sentences = books["description"][i].split(".")  # Split descs. into sentences
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)

    for label in emotion_labels:
        emotions_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


- We will now have a dict which will be the basis of our new df containing 7 diff cols with max prob's corresponding to each individual book

In [24]:
emotions_scores

{'anger': [np.float64(0.0641336739063263),
  np.float64(0.6126186847686768),
  np.float64(0.0641336739063263),
  np.float64(0.35148343443870544),
  np.float64(0.08141246438026428),
  np.float64(0.23222483694553375),
  np.float64(0.5381841063499451),
  np.float64(0.0641336739063263),
  np.float64(0.30066993832588196),
  np.float64(0.0641336739063263)],
 'disgust': [np.float64(0.27359238266944885),
  np.float64(0.34828537702560425),
  np.float64(0.10400671511888504),
  np.float64(0.1507224440574646),
  np.float64(0.18449558317661285),
  np.float64(0.7271748185157776),
  np.float64(0.1558550000190735),
  np.float64(0.10400671511888504),
  np.float64(0.2794812023639679),
  np.float64(0.1779278963804245)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.9425278306007385),
  np.float64(0.9723208546638489),
  np.float64(0.36070698499679565),
  np.float64(0.09504340589046478),
  np.float64(0.051362842321395874),
  np.float64(0.7474278211593628),
  np.float64(0.40449532866477966),
  np.

In [25]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"] # cols of new df
isbn = [] # key for merging to df
emotions_scores = {label: [] for label in emotion_labels}    # Extract all scores for every desc. for each label

# All books
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])  # add individual isbns of each book to the list
    sentences = books["description"][i].split(".")  # Split descs. into sentences
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)

    for label in emotion_labels:
        emotions_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:01<00:00, 42.78it/s]


In [26]:
# Pass the results into a pandas df
emotions_df = pd.DataFrame(emotions_scores)

#  Create a separate col for isbn
emotions_df["isbn13"] = isbn

In [27]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928169,0.932798,0.646216,0.967158,0.729603,9780002005883
1,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078766,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475881,0.078766,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078766,9788185300535


- We now see in our df , we got separate cols for each emotion category, with max probs for each desc.

In [28]:
# Merge the results to our original books df
books = pd.merge(books, emotions_df, on="isbn13")

In [29]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273592,0.928169,0.932798,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150722,0.360707,0.251881,0.732685,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184496,0.095043,0.040564,0.884390,0.475881,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078766


In [30]:
books.to_csv("books_with_emotion_final_df.csv", index=False)